https://zenn.dev/ohno/articles/0ba7970d419898

In [1]:
versioninfo()

Julia Version 1.8.0-beta1
Commit 7b711ce699 (2022-02-23 15:09 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 12 × Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 12 on 12 virtual cores
Environment:
  JULIA_DEPOT_PATH = D:\.julia
  JULIA_NUM_THREADS = 12
  JULIA_PYTHONCALL_EXE = D:\.julia\conda\3\python.exe


In [2]:
using BenchmarkTools

In [3]:
function bad(a)
    s = zero(eltype(a))
    for i in axes(a, 1)
        for j in axes(a, 2)
           s += a[i,j]
        end
    end
    s
end

bad (generic function with 1 method)

In [4]:
function good(a)
    s = zero(eltype(a))
    for j in axes(a, 2)
        for i in axes(a, 1)
           s += a[i,j]
        end
    end
    s
end

good (generic function with 1 method)

In [5]:
function better(a)
    s = zero(eltype(a))
    for j in axes(a, 2)
        @simd for i in axes(a, 1)
           s += a[i,j]
        end
    end
    s
end

better (generic function with 1 method)

In [6]:
function better2(a)
    s = zero(eltype(a))
    @simd for k in keys(a)
        s += a[k]
    end
    s
end

better2 (generic function with 1 method)

In [7]:
function better3(a)
    s = zero(eltype(a))
    @simd for x in a
        s += x
    end
    s
end

better3 (generic function with 1 method)

In [8]:
A = rand(4000, 4000);

In [9]:
@btime bad($A)

  80.982 ms (0 allocations: 0 bytes)


8.0018488062925525e6

In [10]:
@btime good($A)

  15.386 ms (0 allocations: 0 bytes)


8.0018488062907215e6

In [11]:
@btime better($A)

  5.925 ms (0 allocations: 0 bytes)


8.001848806290694e6

In [12]:
@btime better2($A)

  5.930 ms (0 allocations: 0 bytes)


8.001848806290694e6

In [13]:
@btime better3($A)

  5.304 ms (0 allocations: 0 bytes)


8.001848806290893e6

In [14]:
@btime sum($A)

  6.053 ms (0 allocations: 0 bytes)


8.001848806290921e6

In [15]:
keys(rand(5, 3))

CartesianIndices((5, 3))

In [16]:
function good_original(arr)
    sum = 0
    for j in 1:size(arr)[2]
        for i in 1:size(arr)[1]
           sum += arr[i,j]
        end
    end
    return sum
end

@code_warntype good_original(A)

MethodInstance for good_original(::Matrix{Float64})
  from good_original(arr) in Main at In[16]:1
Arguments
  #self#::Core.Const(good_original)
  arr::Matrix{Float64}
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  sum::Union{Float64, Int64}
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  j::Int64
  i::Int64
Body::Union{Float64, Int64}
1 ─       (sum = 0)
│   %2  = Main.size(arr)::Tuple{Int64, Int64}
│   %3  = Base.getindex(%2, 2)::Int64
│   %4  = (1:%3)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%4))
│   %6  = (@_3 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #7 if not %7
2 ┄ %9  = @_3::Tuple{Int64, Int64}
│         (j = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = Main.size(arr)::Tuple{Int64, Int64}
│   %13 = Base.getindex(%12, 1)::Int64
│   %14 = (1:%13)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_5 = Base.iterate(%14))
│   %16 = (@_5 === nothing)::Bo